install libraries

In [10]:
# from google.colab import drive
# drive.mount('Oppositional_thinking')

In [2]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00a 0:00:010m


In [3]:
# test gpu:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
torch.cuda.device_count()

cuda


2

In [6]:
import os
import torch
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam, RMSprop
from transformers  import  get_scheduler
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import KFold
import wandb
from datetime import datetime
import os
import shutil
import random
import numpy as np
from itertools import product


utils

In [7]:
# utils.py
def set_seed(seed):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)

    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

def remove_previous_model(folder):
    dirs = [x for x in os.listdir(folder) if os.path.isdir(folder+os.sep+x)]
    for x in dirs:
        shutil.rmtree(folder+os.sep+x, ignore_errors=False, onerror=None)


def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in product(*vals):
        yield dict(zip(keys, instance))

# mydataset.py
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, mode='train'):
        self.encodings = encodings
        if mode !="train":
            self.labels=  [0]*len(encodings)
        else: self.labels = labels


    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

# datareader.py
import pandas as pd
import json

BINARY_MAPPING_CRITICAL_POS = {'CONSPIRACY': 0, 'CRITICAL': 1}
BINARY_MAPPING_CONSPIRACY_POS = {'CRITICAL': 0, 'CONSPIRACY': 1}

CATEGORY_MAPPING_CRITICAL_POS_INVERSE = {0: 'CONSPIRACY', 1: 'CRITICAL'}
CATEGORY_MAPPING_CONSPIRACY_POS_INVERSE = {0: 'CRITICAL', 1: 'CONSPIRACY'}

TRAIN_DATASET_ES="/kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_es_train.json"
TRAIN_DATASET_EN="/kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_en_train.json"
#TEST_DATASET_EN ="./dataset_oppositional/test/dataset_oppositional_test_nolabels/dataset_en_official_test_nolabels.json"
#TEST_DATASET_ES ="./dataset_oppositional/test/dataset_oppositional_test_nolabels/dataset_en_official_test_nolabels.json"


class PAN24Reader:
    def __init__(self):
        pass
    def read_json_file(self, path):
        dataset=[]
        print(f'Loading official JSON {path} dataset')
        with open(path, 'r', encoding='utf-8') as file:
            dataset = json.load(file)
        return dataset

    def load_dataset_classification(self, path, string_labels=False, positive_class='conspiracy'):
        dataset = self.read_json_file(path)
        # convert to a format suitable for classification
        texts = pd.Series([doc['text'] for doc in dataset])
        if string_labels:
            classes = pd.Series([doc['category'] for doc in dataset])
        else:
            if positive_class == 'conspiracy':
                binmap = BINARY_MAPPING_CONSPIRACY_POS
            elif positive_class == 'critical':
                binmap = BINARY_MAPPING_CRITICAL_POS
            else:
                raise ValueError(f'Unknown positive class: {positive_class}')
            classes = [binmap[doc['category']] for doc in dataset]
            classes = pd.Series(classes)
        ids = pd.Series([doc['id'] for doc in dataset])
        data = pd.DataFrame({"text": texts, "id": ids, "label": classes})
        return data


myReader=PAN24Reader()
es_train_df = myReader.load_dataset_classification(TRAIN_DATASET_ES, string_labels=False, positive_class='conspiracy')
en_train_df = myReader.load_dataset_classification(TRAIN_DATASET_EN, string_labels=False, positive_class='conspiracy')



Loading official JSON /kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_es_train.json dataset
Loading official JSON /kaggle/input/dataset-oppositional/dataset_oppositional/training/dataset_oppositional/dataset_en_train.json dataset


fine_tunning.py

In [9]:
def training(_wandb, _model, _train_data, _val_data, _learning_rate, _optimizer_name, _schedule, _epochs,
             _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True,
             _patience=10, _measure= "accuracy", _out=None):
    train_encodings = _tokenizer(_train_data["text"].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    val_encodings = _tokenizer(_val_data["text"].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")

    train, val = MyDataset(train_encodings, _train_data["label"].tolist()), MyDataset(val_encodings, _val_data["label"].tolist())

    train_dataloader, val_dataloader  = torch.utils.data.DataLoader(train, batch_size=_batch_size, shuffle=True), torch.utils.data.DataLoader(val, batch_size=_batch_size)


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    _wandb.log({"divice": str(device)})
    if use_cuda:
        model = _model.to(device)
        if torch.cuda.device_count() > 1:
            print(f"Usando {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model)
    best_measure, best_model_name, patience = None, None, 0
    training_stats = []
    
    # train_eval = evaluate.load("accuracy")
    train_eval = evaluate.load(f"Yeshwant123/{_measure}")

    lr_scheduler, optimizer = None, None
    #Here we can specify different methods to optmize the paarameters, initially we can consider Adam and RmsProp

    _wandb.log({"info": "Creating the Optimizer and Schedule "})

    lr_scheduler, optimizer = None, None
    if _optimizer_name == "adam":
        optimizer = Adam(_model.parameters(), lr=_learning_rate)
    elif _optimizer_name == "rmsprop":
        optimizer = RMSprop(_model.parameters(), lr=_learning_rate)

    #Here we can define different learning rate schedules, to variate de learning rate in each training step. Initially we use
    # can use linear learning rate schedule
    num_training_steps = _epochs * len(_train_data)
    if _schedule=="linear":
        lr_scheduler = get_scheduler(_schedule, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


    for epoch in range(_epochs):
        if patience >= _patience: break
        total_loss_train, total_acc_train = 0, 0
        total_train_step = 0
        
        _model.train()
        
        for batch in train_dataloader:
            total_train_step += 1
            # print("Epoch ", epoch, "Batch", i)
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            if loss.dim() > 0:
                loss = loss.mean()  # Ensure the loss is a scalar
            loss.backward()
            total_loss_train += loss.item()
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            train_eval.add_batch(predictions=predictions, references=batch["labels"])
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        total_acc_train = train_eval.compute()

        total_eval_steps = 0
        total_loss_val, total_acc_val = 0, 0
        eval_metric = evaluate.load(f"Yeshwant123/{_measure}")
        model.eval()
        for batch in val_dataloader:
            total_eval_steps += 1
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
                loss = outputs.loss
                if loss.dim() > 0:
                    loss = loss.mean()  # Ensure the loss is a scalar
                total_loss_val += loss.item()
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
            eval_metric.add_batch(predictions=predictions, references=batch["labels"])
        total_acc_val = eval_metric.compute()

        if best_measure is None or (best_measure < total_acc_val[_measure]):  # here you must set your save weights
            if best_measure == None: _wandb.log({"info": "It's the first time (epoch) ******************"})
            elif best_measure < total_acc_val[_measure]:
                _wandb.log({"info": "In this epoch an improvement was achieved. (epoch) ******************"})

            best_measure = total_acc_val[_measure]
            try:
                os.makedirs(_out + os.sep + 'models', exist_ok=True)
            except OSError as error:
                _wandb.log({"info": "Directory '%s' can not be created"})
            # remove the directories
            remove_previous_model(_out + os.sep + 'models')
            best_model_name = _out + os.sep + 'models/bestmodel_epoch_{}'.format(epoch +1)
            _wandb.log({"info": "The current best model is " + best_model_name + " "+ str(best_measure)})

            os.makedirs(best_model_name, exist_ok=True)
            if isinstance(model, torch.nn.DataParallel):
                model.module.save_pretrained(best_model_name)
            else:
                model.save_pretrained(best_model_name)
            patience = 0
        else:
            patience += 1
        training_stats.append(
            {
                'epoch': epoch + 1,
                'Training Loss': total_loss_train / total_train_step,
                'Valid. Loss': total_loss_val / total_eval_steps,
                f'Valid.{_measure}': total_acc_val[_measure],
                f'Training.{_measure}': total_acc_train[_measure]
            }
        )
        
        _wandb.log({
            'epoch': epoch + 1,
            'train_loss': total_loss_train / len(train_dataloader),
            f'train_{_measure}': total_acc_train[_measure],
            'val_loss': total_loss_val / len(val_dataloader),
            f'val_{_measure}': total_acc_val[_measure]
        })

    if best_model_name != None:
        if isinstance(model, torch.nn.DataParallel):
            model = model.module.from_pretrained(best_model_name)
        else:
            model = model.from_pretrained(best_model_name)
        _wandb.log({"info": "The final model used to predict the labels of the testing datasets is " + best_model_name})

    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')
    df_stats.to_csv(_out + os.sep + "training_stats.csv")

    _wandb.log({"info": df_stats})
    myplot = sns.lineplot(data=df_stats, palette="tab10", linewidth=2.5)
    fig = myplot.get_figure()
    fig.savefig(_out + os.sep + 'loss-figue.png')
    plt.close()
    return model

############################################################################################################################################################################3
#VALIDATION ON THE TEST SET

def validate(_wandb, _model, _test_data, _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True, _measure="accuracy", evaltype=True):
    test_encodings = _tokenizer(_test_data['text'].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    _mode = "train" if evaltype else "test"
    test = MyDataset(test_encodings, _test_data["label"].tolist(), mode=_mode)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=_batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = _model.to(device)
        if torch.cuda.device_count() > 1:
            print(f"Usando {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model)
    
    eval_metric, out, k = None, None, 0

    if evaltype==True:
        eval_metric = evaluate.load(f"Yeshwant123/{_measure}")

    model.eval()
    total_loss = 0

    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
                outputs = _model(**batch)
                loss = outputs.loss
                if loss.dim() > 0:
                    loss = loss.mean()  # Ensure the loss is a scalar
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                if k == 0:
                    out = predictions
                else:
                    out = torch.cat((out, predictions), 0)
                k += 1
                total_loss += loss.item()
                if evaltype:
                    eval_metric.add_batch(predictions=predictions, references=batch["labels"])

    if evaltype==True:
        total_acc_test = eval_metric.compute()
        test_mesure = total_acc_test[_measure]
        avg_test_loss = total_loss / len(test_dataloader)        # Log the test accuracy and loss to wandb
        _wandb.log({
            f'test_{_measure}': test_mesure,
            'test_avg_loss': avg_test_loss
        })
    return out


In [20]:
# hyperparams_covered = [
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 5, 
#         "epochs": 5,
#         "measure": "mcc",
#         "batch_size": 32, 
#         "max_length": 128
#     },
#     {
#         "optimizer_name": "adam", 
#         "learning": 0.5e-5, 
#         "schedule": "linear",
#         "patience": 5, 
#         "epochs": 10,
#         "measure": "mcc",
#         "batch_size": 32, 
#         "max_length": 128
#     },
# ]
hyperparams_covered = []

main.py

In [17]:
# # Get current date and time
# current_datetime = datetime.now()

# # Format it to include hours, minutes, and seconds
# formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

# settings = wandb.Settings(start_method='thread', console='auto', mode='online')


# wandb.login()

# SEED=1234
# set_seed(SEED)

# preconfig = {
#     0: {
#         "lang": "english",
#         "model_name": "roberta-base",
#     },
#     1: {
#         "lang": "english",
#         "model_name": "microsoft/deberta-base",
#     },
#     # 2: {
#     #     "lang": "spanish",
#     #     "model_name": "dccuchile/bert-base-spanish-wwm-uncased",
#     # },
#     # 3: {
#     #     "lang": "spanish",
#     #     "model_name": "PlanTL-GOB-ES/roberta-base-bne",
#     # },
#     # 4: {
#     #     "lang": "spanish",
#     #     "model_name": "bert-base-multilingual-uncased"
#     # }
# }

# hyperparams = {
#     "optimizer_name": ["adam", "rmsprop"], # ["adam", "rmsprop", "sgd"]
#     "learning": [0.5e-5, 1e-6], # [0.5e-5, 1e-5, 0.5e-6, 1e-6
#     "schedule": ["linear", "cosine"], # ["linear", "cosine", "constant"]
#     "patience": [5, 10], # [3, 5, 10]
#     "epochs": [5, 10], # [5, 10, 20]
#     "measure": ["mcc"],
#     "batch_size": [64], # [16, 32, 64, 128]
#     "max_length": [128]
# }

# # Define KFold cross-validation
# kf = KFold(n_splits=5)

# # For each preconfiguration
# for i, preconfig in preconfig.items():
#     lang = preconfig["lang"]
#     model_name = preconfig["model_name"]
    
#     if lang == "spanish":
#         X= es_train_df
#     elif lang == "english":
#         X= en_train_df
    
#     print("Loading Tokenizer " + model_name)
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     # Initialize a counter for the runs
#     run_counter = 0

#     # For each hyperparameter configuration
#     for config in product_dict(**hyperparams):
#         run_counter += 1
#         if config in hyperparams_covered:
#             continue
#         print(f"Iniciando proceso con los hiperparametros:\n{config}")
            
#         # For each fold
#         for fold, (train_index, val_index) in enumerate(kf.split(X)):
#             X_train, X_val = X.iloc[train_index], X.iloc[val_index]

#             # Start a child run for this fold
#             with wandb.init(project=f'lnr_oppositional_thinking_{formatted_datetime}',
#                             entity='davidandreuroqueta',
#                             group=f'{lang}_{model_name}',
#                             job_type=f'hyperparam-tuning-{run_counter}',
#                             name=f'{lang}_{model_name}_{run_counter}_fold_{fold}',
#                             settings=settings) as fold_run:
#                 fold_run.config.update(preconfig)
#                 fold_run.config.update(config)
#                 fold_run.config.update({"SEED":SEED})

#                 # Log the fold number
#                 fold_run.config.update({"fold": fold + 1})
#                 print(f'Fold: {fold+1}')

#                 wandb.log({"info": "Loading Transformer Model " + model_name})
#                 model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
                

#                 # Train and validate your model, log metrics, etc.
#                 # ...
#                 # FINE-TUNING the model and obtaining the best model across all epochs
#                 fineTmodel = training(_wandb=fold_run, _model=model, _train_data=X_train, _val_data=X_val,
#                                     _learning_rate=config["learning"], _optimizer_name=config["optimizer_name"],
#                                     _schedule=config["schedule"], _epochs=config["epochs"], _tokenizer=tokenizer,
#                                     _batch_size=config["batch_size"], _padding="max_length", _max_length=config["max_length"],
#                                     _truncation=True, _patience=config["patience"], _measure=config["measure"], _out="./out")

#                 # VALIDATING OR PREDICTING on the test partition, this time I'm using the validation set, but you have to use the test set.
#                 preds = validate(_wandb=fold_run, _model=fineTmodel, _test_data=X_val, _tokenizer=tokenizer,
#                                 _batch_size=config["batch_size"], _padding="max_length", _max_length=config["max_length"],
#                                 _truncation=True, _measure=config["measure"], evaltype=True)
                
#                 # Save the fine-tuned model to WandB
#                 model_artifact = wandb.Artifact(f'{lang}_{model_name.replace("/", "-")}_run_{run_counter}_fold_{fold}_model', type='model')
#                 model_save_path = f'./models/{lang}_{model_name.replace("/", "-")}_run_{run_counter}_fold_{fold}'
#                 os.makedirs(model_save_path, exist_ok=True)
#                 fineTmodel.save_pretrained(model_save_path)
#                 tokenizer.save_pretrained(model_save_path)
#                 model_artifact.add_dir(model_save_path)
#                 fold_run.log_artifact(model_artifact)

#                 # Save predictions and corresponding test values to WandB
#                 preds_save_path = f"./predictions/{lang}_{model_name.replace("/", "-")}_run_{run_counter}_fold_{fold}.pkl"
#                 os.makedirs(os.path.dirname(preds_save_path), exist_ok=True)
#                 with open(preds_save_path, "wb") as f:
#                     pickle.dump({"predictions": preds, "test_values": X_val}, f)
#                 preds_artifact = wandb.Artifact(f'{lang}_{model_name.replace("/", "-")}_run_{run_counter}_fold_{fold}_predictions', type='predictions')
#                 preds_artifact.add_file(preds_save_path)
#                 fold_run.log_artifact(preds_artifact)

#     # End the parent run
#     # parent_run.finish()


In [ ]:
import os
import pickle
import numpy as np
import wandb
from datetime import datetime
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import matthews_corrcoef
import torch
from sklearn.model_selection import train_test_split


# Get current date and time
current_datetime = datetime.now()

# Format it to include hours, minutes, and seconds
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

settings = wandb.Settings(start_method='thread', console='auto', mode='online')


wandb.login()

SEED=1234
set_seed(SEED)

preconfig = {
    0: {
        "lang": "english",
        "model_name": "roberta-base",
    },
#     1: {
#         "lang": "english",
#         "model_name": "microsoft/deberta-base",
#     },
    # 2: {
    #     "lang": "spanish",
    #     "model_name": "dccuchile/bert-base-spanish-wwm-uncased",
    # },
    # 3: {
    #     "lang": "spanish",
    #     "model_name": "PlanTL-GOB-ES/roberta-base-bne",
    # },
    # 4: {
    #     "lang": "spanish",
    #     "model_name": "bert-base-multilingual-uncased"
    # }
}

hyperparams = {
    "optimizer_name": ["rmsprop", "rmsprop"], # ["adam", "rmsprop", "sgd"]
    "learning": [0.5e-5], # [0.5e-5, 1e-5, 0.5e-6, 1e-6
    "schedule": ["linear"], # ["linear", "cosine", "constant"]
    "patience": [5, 10], # [3, 5, 10]
    "epochs": [10, 20], # [5, 10, 20]
    "measure": ["mcc"],
    "batch_size": [64], # [16, 32, 64, 128]
    "max_length": [128]
}

# Define KFold cross-validation
kf = KFold(n_splits=5)

# For each preconfiguration
for i, preconfig in preconfig.items():
    lang = preconfig["lang"]
    model_name = preconfig["model_name"]
    
    if lang == "spanish":
        X= es_train_df
    elif lang == "english":
        X= en_train_df
        
    # Split 20% of the data into a separate dataset
    X_train, test_20_percent = train_test_split(X, test_size=0.20, random_state=SEED)
    
    # Create directory if it does not exist
    directory = f'data/test_20_percent_data_{lang}_{model_name}'
    os.makedirs(directory, exist_ok=True)
    
    # Save the 20% dataset to a CSV file
    csv_path = os.path.join(directory, 'data.csv')
    test_20_percent.to_csv(csv_path, index=False)
    
    X = X_train
    print("Loading Tokenizer " + model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Initialize a counter for the runs
    run_counter = 0

    # For each hyperparameter configuration
    for config in product_dict(**hyperparams):
        run_counter += 1
        if config in hyperparams_covered:
            continue
        print(f"Iniciando proceso con los hiperparametros:\n{config}")
            
        # For each fold
        for fold, (train_index, val_index) in enumerate(kf.split(X)):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]

            # Start a child run for this fold
            with wandb.init(project=f'lnr_oppositional_thinking_{formatted_datetime}',
                            entity='davidandreuroqueta',
                            group=f'{lang}_{model_name}',
                            job_type=f'hyperparam-tuning-{run_counter}',
                            name=f'{lang}_{model_name}_{run_counter}_fold_{fold}',
                            settings=settings) as fold_run:
                fold_run.config.update(preconfig)
                fold_run.config.update(config)
                fold_run.config.update({"SEED":SEED})

                # Log the fold number
                fold_run.config.update({"fold": fold + 1})
                print(f'Fold: {fold+1}')

                wandb.log({"info": "Loading Transformer Model " + model_name})
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
                

                # Train and validate your model, log metrics, etc.
                # ...
                # FINE-TUNING the model and obtaining the best model across all epochs
                fineTmodel = training(_wandb=fold_run, _model=model, _train_data=X_train, _val_data=X_val,
                                    _learning_rate=config["learning"], _optimizer_name=config["optimizer_name"],
                                    _schedule=config["schedule"], _epochs=config["epochs"], _tokenizer=tokenizer,
                                    _batch_size=config["batch_size"], _padding="max_length", _max_length=config["max_length"],
                                    _truncation=True, _patience=config["patience"], _measure=config["measure"], _out="./out")

                # VALIDATING OR PREDICTING on the test partition, this time I'm using the validation set, but you have to use the test set.
                preds = validate(_wandb=fold_run, _model=fineTmodel, _test_data=X_val, _tokenizer=tokenizer,
                                _batch_size=config["batch_size"], _padding="max_length", _max_length=config["max_length"],
                                _truncation=True, _measure=config["measure"], evaltype=True)
                
                # Save the fine-tuned model to WandB
                model_artifact = wandb.Artifact(f'{lang}_{model_name}_run_{run_counter}_fold_{fold}_model', type='model')
                model_save_path = f'./models/{lang}_{model_name}_run_{run_counter}_fold_{fold}'
                os.makedirs(model_save_path, exist_ok=True)
                fineTmodel.save_pretrained(model_save_path)
                tokenizer.save_pretrained(model_save_path)
                model_artifact.add_dir(model_save_path)
                fold_run.log_artifact(model_artifact)

                # Save predictions and corresponding test values to WandB
                preds_save_path = f"./predictions/{lang}_{model_name}_run_{run_counter}_fold_{fold}.pkl"
                os.makedirs(os.path.dirname(preds_save_path), exist_ok=True)
                with open(preds_save_path, "wb") as f:
                    pickle.dump({"predictions": preds, "test_values": X_val}, f)
                preds_artifact = wandb.Artifact(f'{lang}_{model_name}_run_{run_counter}_fold_{fold}_predictions', type='predictions')
                preds_artifact.add_file(preds_save_path)
                fold_run.log_artifact(preds_artifact)

    # End the parent run
    # parent_run.finish()


Loading Tokenizer roberta-base
Iniciando proceso con los hiperparametros:
{'optimizer_name': 'rmsprop', 'learning': 5e-06, 'schedule': 'linear', 'patience': 5, 'epochs': 10, 'measure': 'mcc', 'batch_size': 64, 'max_length': 128}


wandb: Currently logged in as: davidandreuroqueta. Use `wandb login --relogin` to force relogin


Fold: 1


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

epoch,▁▂▃▃▄▅▆▆▇█
test_avg_loss,▁
test_mcc,▁
train_loss,█▅▄▃▃▂▂▁▁▁
train_mcc,▁▅▆▆▇▇████
val_loss,█▃▁▁▃▅▄▅▇▅
val_mcc,▁▅▆▇▆▇████
divice,cuda
epoch,10
test_avg_loss,0.35858
test_mcc,0.77338


Fold: 3


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Usando 2 GPUs


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


BAGGING

In [20]:
import os
import pickle
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import matthews_corrcoef

# Define SEED for reproducibility
SEED = 1234

# Load the saved 20% test data
test_data_paths = {
    "english_roberta-base": "data/extracted_20_percent_data_english_roberta-base/data.csv",
#     "english_microsoft_deberta-base": "data/extracted_20_percent_data_english_microsoft/deberta-base/data.csv",
    # Add paths for other configurations if needed
}

# Assuming y_test is included in the saved CSV files, load them
test_dfs = {key: pd.read_csv(path) for key, path in test_data_paths.items()}
y_test = test_dfs["label"].values  # Adjust according to your label column

# Initialize list to hold predictions for each model
all_fold_predictions = []

# Define preconfig
preconfig = {
    0: {
        "lang": "english",
        "model_name": "roberta-base",
    },
#     1: {
#         "lang": "english",
#         "model_name": "microsoft/deberta-base",
#     }
# }

# Perform voting for each preconfiguration
for i, config in preconfig.items():
    lang = config["lang"]
    model_name = config["model_name"]
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(f'./models/{lang}_{model_name}_run_1_fold_0')

    fold_predictions = []

    for fold in range(5):
        # Load the saved model for each fold
        model_path = f'./models/{lang}_{model_name}_run_{fold+1}_fold_{fold}'
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        model.eval()

        # Tokenize the test data
        test_data = test_dfs[f"{lang}_{model_name.replace('/', '_')}"]
        test_encodings = tokenizer(test_data["feature1"].tolist(), truncation=True, padding=True, max_length=128)
        test_dataset = Dataset.from_dict(test_encodings)
        test_loader = DataLoader(test_dataset, batch_size=64)

        preds = []
        with torch.no_grad():
            for batch in test_loader:
                inputs = {key: val.to(model.device) for key, val in batch.items()}
                outputs = model(**inputs)
                preds.append(outputs.logits.cpu().numpy())
        
        preds = np.concatenate(preds, axis=0)
        fold_predictions.append(preds)

    all_fold_predictions.append(fold_predictions)

# Aggregate predictions by voting
aggregated_preds = np.mean(np.array(all_fold_predictions), axis=0)
final_preds = np.argmax(aggregated_preds, axis=1)

# Evaluate the final predictions
mcc = matthews_corrcoef(y_test, final_preds)

print(f'Bagging Model MCC: {mcc}')

FileNotFoundError: [Errno 2] No such file or directory: 'data/extracted_20_percent_data_english_roberta-base/data.csv'

In [17]:
import os
import pickle
import numpy as np
import wandb
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import matthews_corrcoef

# Define SEED for reproducibility
SEED = 1234

# Initialize WandB
wandb.login()

# Load the saved 20% test data
test_data_paths = {
    "english_roberta-base": "/kaggle/input/lnr-test-data/data.csv",
}

# Assuming y_test is included in the saved CSV files, load them
test_dfs = {key: pd.read_csv(path) for key, path in test_data_paths.items()}
y_test = test_dfs["english_roberta-base"]["label"].values  # Adjust according to your label column

# Initialize list to hold predictions for each model
all_fold_predictions = []

# Define preconfig
preconfig = {
    0: {
        "lang": "english",
        "model_name": "roberta-base",
    },
}

# Perform voting for each preconfiguration
for i, config in preconfig.items():
    lang = config["lang"]
    model_name = config["model_name"]
    
    # Initialize WandB run context
    run_name = f"{lang}_{model_name}"
    api = wandb.Api()

    fold_predictions = []

    for fold in range(5):
        # Load the model artifact from WandB
        artifact = api.artifact(f'davidandreuroqueta/lnr_oppositional_thinking_2024-05-27_08-02-11/{run_name}_run_1_fold_{fold}_model:latest')
        artifact_dir = artifact.download()
        model_path = artifact_dir

        # Load the model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model.eval()

        # Tokenize the test data
        test_data = test_dfs[f"{lang}_{model_name.replace('/', '_')}"]
        test_encodings = tokenizer(test_data["feature1"].tolist(), truncation=True, padding=True, max_length=128)
        test_dataset = Dataset.from_dict(test_encodings)
        test_loader = DataLoader(test_dataset, batch_size=64)

        preds = []
        with torch.no_grad():
            for batch in test_loader:
                inputs = {key: val.to(model.device) for key, val in batch.items()}
                outputs = model(**inputs)
                preds.append(outputs.logits.cpu().numpy())
        
        preds = np.concatenate(preds, axis=0)
        fold_predictions.append(preds)

    all_fold_predictions.append(fold_predictions)

# Aggregate predictions by voting
aggregated_preds = np.mean(np.array(all_fold_predictions), axis=0)
final_preds = np.argmax(aggregated_preds, axis=1)

# Evaluate the final predictions
mcc = matthews_corrcoef(y_test, final_preds)

print(f'Bagging Model MCC: {mcc}')

wandb: Downloading large artifact english_roberta-base_run_1_fold_0_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.7


KeyError: 'feature1'

In [14]:
os.path.exists('/kaggle/input/lnr-test-data/data.csv')

df = pd.read_csv('/kaggle/input/lnr-test-data/data.csv')
df['label'].values

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,

In [23]:
import os
import numpy as np
import wandb
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
from sklearn.metrics import matthews_corrcoef, accuracy_score
from datasets import Dataset as HFDataset

# Define SEED for reproducibility
SEED = 1234

# Initialize WandB
wandb.login()

# Load the saved 20% test data
test_data_paths = {
    "english_roberta-base": "/kaggle/input/lnr-test-data/data.csv",
}

# Assuming y_test is included in the saved CSV files, load them
test_dfs = {key: pd.read_csv(path) for key, path in test_data_paths.items()}
y_test = test_dfs["english_roberta-base"]["label"].values  # Adjust according to your label column

# Initialize list to hold predictions for each model
all_fold_predictions = []

# Define preconfig
preconfig = {
    0: {
        "lang": "english",
        "model_name": "roberta-base",
    },
}

# Perform voting for each preconfiguration
for i, config in preconfig.items():
    lang = config["lang"]
    model_name = config["model_name"]
    
    # Initialize WandB run context
    run_name = f"{lang}_{model_name}"
    api = wandb.Api()

    fold_predictions = []

    for fold in range(5):
        # Load the model artifact from WandB
        artifact = api.artifact(f'davidandreuroqueta/lnr_oppositional_thinking_2024-05-27_08-02-11/{run_name}_run_1_fold_{fold}_model:latest')
        artifact_dir = artifact.download()
        model_path = artifact_dir

        # Load the model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model.eval()

        # Tokenize the test data
        test_data = test_dfs[f"{lang}_{model_name.replace('/', '_')}"]
        
        # List columns to verify the correct column name
        print(f"Columns in {lang}_{model_name.replace('/', '_')}: {test_data.columns.tolist()}")
        
        # Adjust column name if necessary
        column_name = 'text'  # Change this to the correct column name based on the printed columns
        if column_name not in test_data.columns:
            raise ValueError(f"Column '{column_name}' not found in test data columns: {test_data.columns.tolist()}")
        
        test_encodings = tokenizer(test_data[column_name].tolist(), truncation=True, padding=True, max_length=128)
        test_dataset = HFDataset.from_dict(test_encodings)
        
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        test_loader = DataLoader(test_dataset, batch_size=64, collate_fn=data_collator)

        preds = []
        with torch.no_grad():
            for batch in test_loader:
                inputs = {key: val.to(model.device) for key, val in batch.items()}
                outputs = model(**inputs)
                preds.append(outputs.logits.cpu().numpy())
        
        preds = np.concatenate(preds, axis=0)
        fold_predictions.append(preds)

    all_fold_predictions.append(fold_predictions)

# Aggregate predictions by voting
aggregated_preds = np.mean(np.array(all_fold_predictions), axis=0)
final_preds = np.argmax(aggregated_preds, axis=1)

# Evaluate the final predictions
mcc = matthews_corrcoef(y_test, final_preds)

print(f'Bagging Model MCC: {mcc}')


2024-05-27 12:04:42.350529: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 12:04:42.350631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 12:04:42.511160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Downloading large artifact english_roberta-base_run_1_fold_0_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:1.0


Columns in english_roberta-base: ['text', 'id', 'label']


wandb: Downloading large artifact english_roberta-base_run_1_fold_1_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.1


Columns in english_roberta-base: ['text', 'id', 'label']


wandb: Downloading large artifact english_roberta-base_run_1_fold_2_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.9


...

wandb: Downloading large artifact english_roberta-base_run_1_fold_3_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:3.6


Columns in english_roberta-base: ['text', 'id', 'label']


wandb: Downloading large artifact english_roberta-base_run_1_fold_4_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:4.3


Columns in english_roberta-base: ['text', 'id', 'label']


ValueError: Found input variables with inconsistent numbers of samples: [800, 5]

In [16]:
import os
import numpy as np
import wandb
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import matthews_corrcoef
from collections import Counter

# Define SEED for reproducibility
SEED = 1234

# Initialize WandB
wandb.login()

# Define the validation function
def validate(_wandb, _model, _test_data, _tokenizer, _batch_size=32, _padding="max_length", _max_length=512, _truncation=True, _measure="accuracy", evaltype=True):
    # Tokenización de los datos de prueba
    test_encodings = _tokenizer(_test_data['text'].tolist(), max_length=_max_length, truncation=_truncation, padding=_padding, return_tensors="pt")
    test = MyDataset(test_encodings, _test_data["label"].tolist())
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=_batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    _model.to(device)  # Mueve el modelo al dispositivo adecuado
    if torch.cuda.device_count() > 1:
        print(f"Usando {torch.cuda.device_count()} GPUs")
        _model = torch.nn.DataParallel(_model)

    if evaltype:
        eval_metric = evaluate.load(f"Yeshwant123/{_measure}")

    _model.eval()
    total_loss = 0
    all_predictions = []

    with torch.no_grad():
        for batch in test_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = _model(**batch)
            loss = outputs.loss
            if loss.dim() > 0:
                loss = loss.mean()  # Asegura que la pérdida es un escalar
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            all_predictions.append(predictions.cpu())  # Move predictions to CPU
            total_loss += loss.item()
            if evaltype:
                eval_metric.add_batch(predictions=predictions, references=batch["labels"])

    # Concatenar todas las predicciones
    all_predictions = torch.cat(all_predictions).cpu().numpy()  # Ensure tensor is on CPU before converting to numpy

    if evaltype:
        total_acc_test = eval_metric.compute()
        test_mesure = total_acc_test[_measure]
        avg_test_loss = total_loss / len(test_dataloader)
        _wandb.log({
            f'test_{_measure}': test_mesure,
            'test_avg_loss': avg_test_loss
        })

    return all_predictions

# Load the saved 20% test data
test_data_paths = {
    "english_roberta-base": "/kaggle/input/lnr-test-data/data.csv",
}

# Assuming y_test is included in the saved CSV files, load them
test_dfs = {key: pd.read_csv(path) for key, path in test_data_paths.items()}
y_test = test_dfs["english_roberta-base"]["label"].values  # Adjust according to your label column

# Initialize matrix to hold predictions for each fold (800 samples x 5 folds)
num_folds = 5
num_samples = len(y_test)
fold_predictions = np.zeros((num_samples, num_folds))

# Define preconfig
preconfig = {
    0: {
        "lang": "english",
        "model_name": "roberta-base",
    },
}

# Perform voting for each preconfiguration
for i, config in preconfig.items():
    lang = config["lang"]
    model_name = config["model_name"]
    
    # Initialize WandB run context
    run_name = f"{lang}_{model_name}"
    api = wandb.Api()

    for fold in range(num_folds):
        # Load the model artifact from WandB
        artifact = api.artifact(f'davidandreuroqueta/lnr_oppositional_thinking_2024-05-27_08-02-11/{run_name}_run_1_fold_{fold}_model:latest')
        artifact_dir = artifact.download()
        model_path = artifact_dir

        # Load the model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model.eval()

        # Tokenize the test data
        test_data = test_dfs[f"{lang}_{model_name.replace('/', '_')}"]
        
        # List columns to verify the correct column name
        print(f"Columns in {lang}_{model_name.replace('/', '_')}: {test_data.columns.tolist()}")
        
        # Adjust column name if necessary
        column_name = 'text'  # Change this to the correct column name based on the printed columns
        if column_name not in test_data.columns:
            raise ValueError(f"Column '{column_name}' not found in test data columns: {test_data.columns.tolist()}")
        
        predictions = validate(_wandb=wandb, _model=model, _test_data=test_data, _tokenizer=tokenizer, _batch_size=64, _padding="max_length", _max_length=128, _truncation=True, _measure="accuracy", evaltype=False)
        
        fold_predictions[:, fold] = predictions

# Compute the majority vote for each sample
final_preds = [Counter(fold_predictions[i, :]).most_common(1)[0][0] for i in range(num_samples)]

# Evaluate the final predictions
mcc = matthews_corrcoef(y_test, final_preds)

print(f'Bagging Model MCC: {mcc}')

wandb: Downloading large artifact english_roberta-base_run_1_fold_0_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:1.0


Columns in english_roberta-base: ['text', 'id', 'label']
Usando 2 GPUs


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
wandb: Downloading large artifact english_roberta-base_run_1_fold_1_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.9


Columns in english_roberta-base: ['text', 'id', 'label']
Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_1_fold_2_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:1.0


Columns in english_roberta-base: ['text', 'id', 'label']
Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_1_fold_3_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.9


Columns in english_roberta-base: ['text', 'id', 'label']
Usando 2 GPUs


wandb: Downloading large artifact english_roberta-base_run_1_fold_4_model:latest, 478.72MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.9


Columns in english_roberta-base: ['text', 'id', 'label']
Usando 2 GPUs
Bagging Model MCC: 0.7514312061446426


In [18]:
from sklearn.metrics import matthews_corrcoef, accuracy_score

accuracy = accuracy_score(y_test, final_preds)

print(f'Bagging Model Accuracy: {accuracy}')


Bagging Model Accuracy: 0.8875
